In [15]:
import torch
from vision_transformer3 import vision_transformer3 

default_kwargs = {
                            # image
                        'image_size': 224,
                        'patch_size': 32, # if '32' in hparams.arch else 16,
                        'mode': 'base',
                        'embed_dim': 512
                    }
print(default_kwargs)
model=vision_transformer3(**default_kwargs)


pretrained_weights = '/shared/js.yun/DINO_sagemaker/ckpts/exp/epoch=48.ckpt'
state_dict = torch.load(pretrained_weights, map_location="cpu")
# checkpoint_key = 'teacher'
checkpoint_key = 'state_dict'

# for i in state_dict:
#     print(i)

if checkpoint_key is not None and checkpoint_key in state_dict:
    print(f"Take key {checkpoint_key} in provided checkpoint dict")
    state_dict = state_dict[checkpoint_key]
    # print(state_dict)

teacher_dict = {}
for k,v in state_dict.items():
    if 'teacher.backbone' in k:
        key = k.replace("teacher.backbone.", "")
        teacher_dict[key] = v

# remove `module.` prefix
state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
# remove `backbone.` prefix induced by multicrop wrapper
state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
msg = model.load_state_dict(teacher_dict, strict=False)
print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))

{'image_size': 224, 'patch_size': 32, 'mode': 'base', 'embed_dim': 512}


Take key state_dict in provided checkpoint dict
Pretrained weights found at /shared/js.yun/DINO_sagemaker/ckpts/exp/epoch=48.ckpt and loaded with msg: <All keys matched successfully>
